<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/14_Model_Validation_in_Python/02_Validation_Basics-learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 2. Validation Basics


In [41]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor # Import RandomForestRegressor




url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/13_Feature_Engineering_for_Machine_Learning_in_Python/dataset/inaugural_speeches.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding 'text'
speech_df = pd.read_csv(url)
numeric_cols = ['Name', 'Inaugural Address', 'Date'] # Assuming these should be numeric if possible
for col in numeric_cols:
    speech_df[col] = pd.to_numeric(speech_df[col], errors='coerce')

display(speech_df.head())

,Name,Inaugural Address,Date,text
0,NaN,NaN,NaN,Fellow-Citizens of the Senate and of the House...
1,NaN,NaN,NaN,Fellow Citizens: I AM again called upon by th...
2,NaN,NaN,NaN,"WHEN it was first perceived, in early times, t..."
3,NaN,NaN,NaN,Friends and Fellow-Citizens: CALLED upon to u...
4,NaN,NaN,NaN,"PROCEEDING, fellow-citizens, to that qualifica..."


### Creating train, test, and validation datasets

In [4]:
url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/14_Model_Validation_in_Python/dataset/tic-tac-toe.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding non-numeric ones
tic_tac_toe = pd.read_csv(url)


display(tic_tac_toe.head())

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


**Create one holdout set**  
Your boss has asked you to create a simple random forest model on the tic_tac_toe dataset. She doesn't want you to spend much time selecting parameters; rather she wants to know how well the model will perform on future data. For future Tic-Tac-Toe games, it would be nice to know if your model can predict which player will win.

The dataset tic_tac_toe has been loaded for your use.

Note that in Python, =\ indicates the code was too long for one line and has been split across two lines.

In [5]:
# Create dummy variables using pandas
X = pd.get_dummies(tic_tac_toe.iloc[:,0:9])
y = tic_tac_toe.iloc[:, 9]

# Create training and testing datasets. Use 10% for the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1111)

# without the holdout set, you cannot truly validate a model. Let's move on to creating two holdout sets.

In [6]:
# Create temporary training and final testing datasets
X_temp, X_test, y_temp, y_test  =\
    train_test_split(X, y, test_size=0.20, random_state=1111)

# Create the final training and validation datasets
X_train, X_val, y_train, y_val  =\
    train_test_split(X_temp, y_temp, test_size=0.25, random_state=1111)

# You now have training, validation, and testing datasets, but do you know _when_ you need both validation and testing datasets? Keep going!
# The next exercise will help make sure you understand when to use validation datasets.

**Mean absolute error**  
Communicating modeling results can be difficult. However, most clients understand that on average, a predictive model was off by some number. This makes explaining the mean absolute error easy. For example, when predicting the number of wins for a basketball team, if you predict 42, and they end up with 40, you can easily explain that the error was two wins.

In this exercise, you are interviewing for a new position and are provided with two arrays. y_test, the true number of wins for all 30 NBA teams in 2017 and predictions, which contains a prediction for each team. To test your understanding, you are asked to both manually calculate the MAE and use sklearn.

In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Convert y_test to numeric and handle NaNs
y_test_numeric = pd.to_numeric(y_test, errors='coerce')

# Step 2: Replace with real predictions from your model, or simulate
predictions = np.random.rand(len(y_test_numeric)) * 10  # MUST match y_test length

# Step 3: Check shapes before proceeding
print("y_test_numeric shape:", y_test_numeric.shape)
print("predictions shape:", predictions.shape)

# Step 4: Mask out NaNs in either array
mask = (~np.isnan(y_test_numeric)) & (~np.isnan(predictions))

# Step 5: Apply mask
y_clean = y_test_numeric[mask]
pred_clean = predictions[mask]

# Step 6: Check if arrays are still valid
if len(y_clean) == 0:
    print("Error: After filtering, no valid data points remain.")
else:
    # Step 7: Compute errors
    mae = mean_absolute_error(y_clean, pred_clean)
    mse = mean_squared_error(y_clean, pred_clean)

    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")


y_test_numeric shape: (30,)
predictions shape: (30,)
MAE: 36.36
MSE: 1426.25


In [23]:
from sklearn.metrics import mean_squared_error

# Actual wins for all 30 NBA teams in 2017
y_test = np.array([
    53, 51, 42, 40, 49, 20, 43, 41, 47, 51,
    31, 28, 45, 47, 33, 40, 42, 48, 33, 31,
    55, 44, 24, 29, 61, 26, 50, 41, 39, 32
])

# Mock predictions from a model
predictions = np.array([
    50, 48, 40, 38, 47, 25, 39, 43, 44, 49,
    35, 30, 42, 45, 30, 38, 44, 46, 35, 33,
    56, 46, 28, 27, 58, 30, 48, 43, 38, 35
])

n = len(predictions)
# Finish the manual calculation of the MSE
mse_one = sum((y_test - predictions)**2) / n
print('With a manual calculation, the error is {}'.format(mse_one))

# Use the scikit-learn function to calculate MSE
mse_two = mean_squared_error(y_test, predictions)
print('Using scikit-learn, the error is {}'.format(mse_two))

# These predictions were about six wins off on average.
# This isn't too bad considering NBA teams play 82 games a year. Let's see how these errors would look
# if you used the mean squared error instead.

With a manual calculation, the error is 7.266666666666667
Using scikit-learn, the error is 7.266666666666667


In [24]:
from sklearn.metrics import mean_absolute_error

# Manually calculate the MAE
n = len(predictions)
mae_one = sum(abs(y_test - predictions)) / n
print('With a manual calculation, the error is {}'.format(mae_one))

mae_two = mean_absolute_error(y_test, predictions)
print('Using scikit-learn, the error is {}'.format(mae_two))

With a manual calculation, the error is 2.533333333333333
Using scikit-learn, the error is 2.533333333333333


**Performance on data subsets**  
In professional basketball, there are two conferences, the East and the West. Coaches and fans often only care about how teams in their own conference will do this year.

You have been working on an NBA prediction model and would like to determine if the predictions were better for the East or West conference. You added a third array to your data called labels, which contains an "E" for the East teams, and a "W" for the West. y_test and predictions have again been loaded for your use.

In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae

# Simulated 30 NBA team names and conference labels
teams = [
    "Celtics", "Cavaliers", "Bulls", "Knicks", "Raptors", "Hawks", "Heat", "Pacers", "Bucks", "Wizards",
    "Warriors", "Spurs", "Thunder", "Clippers", "Rockets", "Jazz", "Grizzlies", "Trail Blazers", "Nuggets", "Pelicans",
    "76ers", "Nets", "Hornets", "Pistons", "Magic", "Kings", "Mavericks", "Lakers", "Timberwolves", "Suns"
]

# East (E) and West (W) labels (first 10 East, rest West for simplicity)
labels = np.array([
    "E", "E", "E", "E", "E", "E", "E", "E", "E", "E",
    "W", "W", "W", "W", "W", "W", "W", "W", "W", "W",
    "E", "E", "E", "E", "E", "W", "W", "W", "W", "W"
])

# Actual 2017 win totals (simulated realistic values)
y_test = np.array([
    53, 51, 42, 40, 49, 20, 43, 41, 47, 51,
    67, 61, 47, 51, 55, 49, 43, 41, 40, 34,
    28, 20, 36, 37, 29, 32, 33, 26, 31, 24
])

# Simulated predictions
predictions = np.array([
    52, 49, 41, 39, 48, 24, 42, 39, 45, 49,
    65, 60, 45, 52, 54, 48, 45, 39, 41, 35,
    30, 22, 34, 36, 30, 35, 32, 28, 33, 27
])

# Get East and West team indices
east_teams = labels == "E"
west_teams = labels == "W"

# Split the true and predicted values
true_east = y_test[east_teams]
preds_east = predictions[east_teams]

true_west = y_test[west_teams]
preds_west = predictions[west_teams]

# Calculate and print MAE
print("The MAE for East teams is:", mae(true_east, preds_east))
print("The MAE for West teams is:", mae(true_west, preds_west))

# It looks like the Western conference predictions were about two games better on average. Over the past few seasons,
# the Western teams have generally won the same number of games as the experts have predicted.
# Teams in the East are just not as predictable as those in the West.


The MAE for East teams is: 1.6666666666666667
The MAE for West teams is: 1.6666666666666667


### Classification matrix

**Confusion matrices**  
Confusion matrices are a great way to start exploring your model's accuracy. They provide the values needed to calculate a wide range of metrics, including sensitivity, specificity, and the F1-score.

You have built a classification model to predict if a person has a broken arm based on an X-ray image. On the testing set, you have the following confusion matrix:

    Prediction: 0	Prediction: 1
    Actual: 0	324 (TN)	15 (FP)
    Actual: 1	123 (FN)	491 (TP)

In [27]:
# Calculate and print the accuracy
accuracy = (324 + 491 ) / (953)
print("The overall accuracy is {0: 0.2f}".format(accuracy))

# Calculate and print the precision
precision = (491) / (491 + 15)
print("The precision is {0: 0.2f}".format(precision))

# Calculate and print the recall
recall = (491) / (491 + 123)
print("The recall is {0: 0.2f}".format(recall))

#  In this case, a true positive is a picture of an actual broken arm that was also predicted to be broken.
#   Doctors are okay with a few additional false positives (predicted broken, not actually broken),
#   as long as you don't miss anyone who needs immediate medical attention.

The overall accuracy is  0.86
The precision is  0.97
The recall is  0.80


**Confusion matrices, again**  
Creating a confusion matrix in Python is simple. The biggest challenge will be making sure you understand the orientation of the matrix. This exercise makes sure you understand the sklearn implementation of confusion matrices. Here, you have created a random forest model using the tic_tac_toe dataset rfc to predict outcomes of 0 (loss) or 1 (a win) for Player One.

Note: If you read about confusion matrices on another website or for another programming language, the values might be reversed.

In [34]:
from sklearn.ensemble import RandomForestClassifier

# Create dummy variables using pandas
X = pd.get_dummies(tic_tac_toe.iloc[:,0:9])
y = tic_tac_toe.iloc[:, 9]

# Create training and testing datasets. Use 10% for the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1111)

# Create a RandomForestClassifier model
rfc = RandomForestClassifier(random_state=1111)

# Train the model using the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=1111)

In [42]:
from sklearn.metrics import confusion_matrix

# Create predictions
test_predictions = rfc.predict(X_test)

# Create and print the confusion matrix
cm = confusion_matrix(y_test, test_predictions)
print(cm)

# Print the true positives (actual 1s that were predicted 1s)
print("The number of true positives is: {}".format(cm[1, 1]))

# Row 1, column 1 represents the number of actual 1s that were predicted 1s (the true positives).
# Always make sure you understand the orientation of the confusion matrix before you start using it!

[[26  4]
 [ 0 66]]
The number of true positives is: 66


**Precision vs. recall**   
The accuracy metrics you use to evaluate your model should always be based on the specific application. For this example, let's assume you are a really sore loser when it comes to playing Tic-Tac-Toe, but only when you are certain that you are going to win.

Choose the most appropriate accuracy metric, either precision or recall, to complete this example. But remember, if you think you are going to win, you better win!

Use rfc, which is a random forest classification model built on the tic_tac_toe dataset.

In [43]:
from sklearn.metrics import precision_score

test_predictions = rfc.predict(X_test)

# Create precision or recall score based on the metric you imported
# Specify 'positive' as the positive class label
score = precision_score(y_test, test_predictions, pos_label='positive')

# Print the final result
print("The precision value is {0:.2f}".format(score))

# Precision is the correct metric here. Sore-losers can't stand losing when they are certain they will win! For that reason,
# our model needs to be as precise as possible. With a precision of only 79%,
# you may need to try some other modeling techniques to improve this score.



The precision value is 0.94


### The Bias=Varience Yradeoff

In [45]:
url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/14_Model_Validation_in_Python/dataset/candy-data.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding non-numeric ones
candy = pd.read_csv(url)


display(candy.head())

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465


In [46]:
# Define features (X) and target (y)
X = candy.drop(['competitorname', 'winpercent'], axis=1)
y = candy['winpercent']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)

In [47]:
# Update the rfr model
rfr = RandomForestRegressor(n_estimators=25,
                            random_state=1111,
                            max_features=2)
rfr.fit(X_train, y_train)

# Print the training and testing accuracies
print('The training error is {0:.2f}'.format(
  mae(y_train, rfr.predict(X_train))))
print('The testing error is {0:.2f}'.format(
  mae(y_test, rfr.predict(X_test))))

The training error is 3.33
The testing error is 9.22


In [48]:
# Update the rfr model
rfr = RandomForestRegressor(n_estimators=25,
                            random_state=1111,
                            max_features=11)
rfr.fit(X_train, y_train)

# Print the training and testing accuracies
print('The training error is {0:.2f}'.format(
  mae(y_train, rfr.predict(X_train))))
print('The testing error is {0:.2f}'.format(
  mae(y_test, rfr.predict(X_test))))

The training error is 3.32
The testing error is 10.01


In [49]:
# Update the rfr model
rfr = RandomForestRegressor(n_estimators=25,
                            random_state=1111,
                            max_features=4)
rfr.fit(X_train, y_train)

# Print the training and testing accuracies
print('The training error is {0:.2f}'.format(
  mae(y_train, rfr.predict(X_train))))
print('The testing error is {0:.2f}'.format(
  mae(y_test, rfr.predict(X_test))))

# The chart below shows the performance at various max feature values. Sometimes, setting parameter values can make a huge difference in model performance.

The training error is 3.23
The testing error is 9.17


**Am I underfitting?**   
You are creating a random forest model to predict if you will win a future game of Tic-Tac-Toe. Using the tic_tac_toe dataset, you have created training and testing datasets, X_train, X_test, y_train, and y_test.

You have decided to create a bunch of random forest models with varying amounts of trees (1, 2, 3, 4, 5, 10, 20, and 50). The more trees you use, the longer your random forest model will take to run. However, if you don't use enough trees, you risk underfitting. You have created a for loop to test your model at the different number of trees.

In [51]:
url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/14_Model_Validation_in_Python/dataset/tic-tac-toe.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding non-numeric ones
tic_tac_toe = pd.read_csv(url)


display(tic_tac_toe.head())

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [52]:
# Create dummy variables using pandas
X = pd.get_dummies(tic_tac_toe.iloc[:,0:9])
y = tic_tac_toe.iloc[:, 9]

# Create training and testing datasets. Use 10% for the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1111)

# without the holdout set, you cannot truly validate a model. Let's move on to creating two holdout sets.

In [53]:
from sklearn.metrics import accuracy_score

test_scores, train_scores = [], []
for i in [1, 2, 3, 4, 5, 10, 20, 50]:
    rfc = RandomForestClassifier(n_estimators=i, random_state=1111)
    rfc.fit(X_train, y_train)
    # Create predictions for the X_train and X_test datasets.
    train_predictions = rfc.predict(X_train)
    test_predictions = rfc.predict(X_test)
    # Append the accuracy score for the test and train predictions.
    train_scores.append(round(accuracy_score(y_train, train_predictions), 2))
    test_scores.append(round(accuracy_score(y_test, test_predictions), 2))
# Print the train and test scores.
print("The training scores were: {}".format(train_scores))
print("The testing scores were: {}".format(test_scores))

# Notice that with only one tree, both the train and test scores are low. As you add more trees, both errors improve.
# Even at 50 trees, this still might not be enough. Every time you use more trees,
# you achieve higher accuracy. At some point though, more trees increase training time, but do not decrease testing error.

The training scores were: [0.93, 0.93, 0.98, 0.98, 0.99, 1.0, 1.0, 1.0]
The testing scores were: [0.75, 0.81, 0.85, 0.92, 0.92, 0.95, 0.95, 0.97]
